In [ ]:
"""
This is a the shrinkage version of Transport map. In this version, 
the regression functions $f_i$ and the niggest parameters $d_i$ are 
assumed to have some specific structures. These values are given by
the parametric covariance matrix. Unlike the example in other notebook,
here we try to estimate the parametric covariance matrix parameters 
using the integrated log-likelihood function.


Author: Anirban Chakraborty,
Last modified: May 13, 2024
"""
%load_ext autoreload
%autoreload 2

### Load necessary libraries

In [ ]:
import torch
import numpy as np
from veccs import orderings
from gpytorch.kernels import MaternKernel
from sklearn.gaussian_process import kernels
from matplotlib import pyplot as plt

from batram.helpers import make_grid, GaussianProcessGenerator
from batram.legmods import Data, SimpleTM
from batram.shrinkmods import ShrinkTM, EstimableShrinkTM


### Comparing log-score with the base transport maps (exponential kernel)

In [ ]:
torch.manual_seed(20240522)

In [ ]:
## kernel and location parameters

num_locs = 30; dim_locs = 2
nu_original = 0.5
length_scale_original = 0.3
numSamples = 30
sd_noise=1e-6
largest_conditioning_set = 30
sigmasq_f = 1.0

In [ ]:
import pickle
with open("../data/NR900ExpLST30SIGSQT10.pkl", "rb") as f:
    data = pickle.load(f)
locs = data["locs"]
locsorder = data["order"]
gp = data["gp"]
torchdata = data["data"][:, locsorder]
nn = orderings.find_nns_l2(locs, largest_conditioning_set)

In [ ]:
## getting the data ready

numSamples = [1, 2, 5, 10, 15, 20, 30, 50, 80, 160, 200]
neglogScore_tm = torch.zeros(len(numSamples))
neglogScore_shrink = torch.zeros(len(numSamples))
tm_models = []
shrink_models = []
yreps = 50 #to be used for estimating log-score
nsteps = 500

In [ ]:
## fit models
for i, n in enumerate(numSamples):
    obs = torchdata[0:n, :] #snip first n samples
    #if obs.dim() == 1:
    #    obs = obs.unsqueeze(0)
    obsTrain = obs

    # Create a `Data` object for use with the `SimpleTM`/ `ShrinkTM` model.
    data_tm = Data.new(torch.as_tensor(locs).float(), obs, torch.as_tensor(nn))
    data_shrink = Data.new(torch.as_tensor(locs).float(), obs, torch.as_tensor(nn))

    tm = SimpleTM(data_tm, theta_init=None, linear=False, smooth=1.5, nug_mult=4.0)
    opt = torch.optim.Adam(tm.parameters(), lr=0.01)
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, nsteps)
    res = tm.fit(
        nsteps, 0.1, test_data=tm.data, optimizer=opt, scheduler=sched, batch_size=300
    )
    tm_models.append(tm)
    
    shrink_tm = EstimableShrinkTM(data=data_shrink, linear=False, 
                     transportmap_smooth=1.5, 
                     parametric_kernel= "exponential",
                     param_nu=0.5,
                     param_ls=1.0,
                     nug_mult_bounded=False)
    opt2 = torch.optim.Adam(shrink_tm.parameters(), lr=0.01)
    sched2 = torch.optim.lr_scheduler.CosineAnnealingLR(opt2, nsteps)
    res2 = shrink_tm.fit(
        nsteps, 0.1, test_data=shrink_tm.data, optimizer=opt2, scheduler=sched2, batch_size=300,

    )
    shrink_models.append(shrink_tm)

    for _j in range(0, 50):
        with torch.no_grad():
            neglogScore_tm[i] += tm.score(torchdata[(200 + _j), :])
        neglogScore_shrink[i] += shrink_tm.score(torchdata[(200 + _j), :])
    

In [ ]:
for i in range(len(shrink_models)):
    with torch.no_grad():
        print(shrink_models[i].parametric_kernel.log_sigmasq.exp().item())

In [ ]:
for i in range(len(shrink_models)):
    with torch.no_grad():
        print(shrink_models[i].parametric_kernel.log_ls.exp().item())

In [ ]:
for i in range(len(shrink_models)):
    with torch.no_grad():
        print(shrink_models[i].transform_shrinkage_factor().item())

In [ ]:
logScore_tm = neglogScore_tm/50
logScore_shrink = neglogScore_shrink/50
plt.plot(torch.arange(len(numSamples)), logScore_tm)
plt.plot(torch.arange(len(numSamples)), logScore_shrink)
plt.ylim(torch.concatenate((logScore_tm, logScore_shrink)).min() - 50, 
         torch.concatenate((logScore_tm, logScore_shrink)).max() + 50)
plt.title("Log-Score of SimpleTM and ShrinkTM")
plt.xticks(torch.arange(len(numSamples)), labels = numSamples);

In [ ]:
plt.plot(torch.arange(len(numSamples)), logScore_shrink)

In [ ]:
torch.save({
    "gp_generator": gp,
    "tm_models": tm_models,
    "shrink_models": shrink_models,
    "tm_logscore" : logScore_tm,
    "shrink_logscore": logScore_shrink,
    "numSamples": numSamples
}, f"../results/modelsNR_LST{int(100*length_scale_original)}_SQT{int(100*sigmasq_f)}.pt")

## Climate data application

In [1]:
import pickle
import math

import torch
import numpy as np
from veccs import orderings
from gpytorch.kernels import MaternKernel
from sklearn.gaussian_process import kernels
from matplotlib import pyplot as plt

from batram.helpers import make_grid, GaussianProcessGenerator
from batram.legmods import Data, SimpleTM
from batram.shrinkmods import ShrinkTM, EstimableShrinkTM

%load_ext autoreload
%autoreload 2

/home/anirban0451/miniconda3/envs/shrink2param/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:

with open("../data/prec_days.pkl", "rb") as f:
    prec_days = pickle.load(f)

#with open("../data/prec_all.pkl", "rb") as f:
#    prec_all = pickle.load(f)

#lat = prec_days["lat"]
#lon = prec_days["lon"]
#obs = np.log(prec_days["precs"][:,:,0].T + 1e-10)
locs = np.loadtxt("../data/locs.csv", skiprows=1, delimiter=",")
lat = locs[:, 1]
lon = locs[:, 0]
obs = np.loadtxt("../data/prec-june1.csv", skiprows=1, delimiter=",").T
#l=lon/360*2*math.pi; L=lat/360*2*math.pi
#locs = (np.vstack([np.cos(L)*np.cos(l),np.cos(L)*np.sin(l),np.sin(L)])).T

In [29]:
#obs_mean = obs.mean(axis = 0, keepdims = True)
obs.mean(axis = 0, keepdims = True)
obs.std(axis = 0, keepdims = True)
#obs_sd = obs.std(axis = 0, keepdims = True)

array([[0.99488488, 0.99488488, 0.99488488, ..., 0.99488488, 0.99488488,
        0.99488488]])

In [18]:
#obs_scaled = (obs - obs_mean)/obs_sd

In [30]:
locsorder = orderings.maxmin_cpp(locs)
locsTrain = locs[locsorder, :]
nn = orderings.find_nns_l2(locs, 30)
shuffle = torch.randperm(obs.shape[0])

obsTrain = torch.from_numpy((obs[shuffle,:])[:, locsorder]).float()

In [31]:
nsteps = 500

In [32]:
data = Data.new(torch.as_tensor(locsTrain).float(), obsTrain[0:3, :], torch.as_tensor(nn))

tm = SimpleTM(data, theta_init=None, linear=False, smooth=1.5, nug_mult=4.0)
opt = torch.optim.Adam(tm.parameters(), lr=0.01)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, nsteps)
res = tm.fit(
    nsteps, 0.1, test_data=tm.data, optimizer=opt, scheduler=sched, batch_size=900
)

shrink_tm = EstimableShrinkTM(data=data, linear=False, 
                    transportmap_smooth=1.5, 
                    parametric_kernel= "exponential",
                    param_nu=0.5,
                    param_ls=0.6,
                    nug_mult_bounded=False)
opt2 = torch.optim.Adam(shrink_tm.parameters(), lr=0.01)
sched2 = torch.optim.lr_scheduler.CosineAnnealingLR(opt2, nsteps)
res2 = shrink_tm.fit(
    nsteps, 0.1, test_data=shrink_tm.data, optimizer=opt2, scheduler=sched2, batch_size=2700,

)

RuntimeError: Failed to compute Cholesky decomposition of G.

In [23]:
neglogScore_shrink = 0
neglogScore_tm = 0
for _j in range(0, 10):
    with torch.no_grad():
        neglogScore_tm += tm.score(obsTrain[(50 + _j), :])
    neglogScore_shrink += shrink_tm.score(obsTrain[(50 + _j), :])

In [24]:
neglogScore_tm, neglogScore_shrink

(tensor(-39102.6250), tensor(-32957.5469))

In [27]:
shrink_tm.parametric_kernel.log_ls.exp()

tensor([4.4907], grad_fn=<ExpBackward0>)